In [7]:
%load_ext cudf.pandas
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
import torch.nn.functional as F
from agent2 import Agents

The cudf.pandas extension is already loaded. To reload it, use:
  %reload_ext cudf.pandas


In [15]:
data = pd.read_csv("/home/uav/Documents/AI_Hunter/LLMS/LLM_simulation_u/sub_para.csv")

In [30]:
data[data["parameter"] =="NUM_CONTROLS"]["description"].to_list()

['uint8 = 8']

In [25]:
df = pd.read_csv("/home/uav/Documents/AI_Hunter/LLMS/ulg_data/06_40_19.csv")

In [32]:
df["s_variance_m_s"].value_counts()

s_variance_m_s
NaN    150874
Name: count, dtype: int64

In [8]:
df= pd.read_csv("/home/uav/Documents/AI_Hunter/LLMS/LLM_simulation_u/user_analytics/i want scenarios that involve very high wind conditions. i wan to test my system more in the context of wind so make sure i cover all edge cases in that conext.csv")

In [12]:
analysis = ""
for res in df["0"]:
    analysis += res

In [14]:
analysis.split("\n")

["Sure, I'd be happy to help you analyze the plots and provide a detailed report on the drone's ULG data. Here's a breakdown of the plots and their potential impact on drone behavior:",
 '',
 '1. Plot of yaw_body:',
 "   - This plot shows the yaw body of the drone, which is a measure of the drone's orientation in relation to the ground. The yaw body is relatively stable, with a few spikes indicating sudden changes in orientation. This could be due to the drone's response to wind or other external factors.",
 '',
 '2. Plot of roll_body:',
 "   - The roll body plot shows the roll of the drone, which is a measure of the drone's tilt in the horizontal plane. The plot has a few spikes, which could be due to the drone's response to wind or other external factors. The roll body is generally stable, indicating that the drone is maintaining a stable flight.",
 '',
 '3. Plot of rollspeed_integ:',
 "   - This plot shows the integral of the roll speed, which is a measure of the drone's roll rate o

In [2]:
Analytics_data = pd.read_csv("/home/uav/Documents/AI_Hunter/LLMS/ulg_data/wind1_context.csv")


In [6]:
Analytics_data[Analytics_data["parameter"]=="mode"]["content"].to_list()

['mode\n description:- uint8 ']

In [ ]:
Analytics_data

In [ ]:
Analytics_data = pd.read_csv("/home/uav/Documents/AI_Hunter/LLMS/LLM_simulation_u/Analytics_knowledge.csv")

In [ ]:
Analytics_data

In [ ]:
images = Agents.load_images_from_folder("i want scenarios that involve very high wind conditions. i wan to test my system more in the context of wind so make sure i cover all edge cases in that conext")

In [ ]:
len(images)

In [ ]:
text,iamges = Agents.Analytics_two("i want scenarios that involve very high wind conditions. i wan to test my system more in the context of wind so make sure i cover all edge cases in that conext")

In [ ]:
len(iamges)

In [ ]:
from transformers import BitsAndBytesConfig

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
images = iamges

In [ ]:
images = Agents.load_images_from_folder(image_path)

In [ ]:
model_id = "microsoft/Phi-3-vision-128k-instruct" 
        # quantization_config = BitsAndBytesConfig(load_in_4bit=True)#,llm_int4_threshold=6.0,llm_int8_skip_modules=None,trust_remote_code=True)
        
model, tokenizer, processor = Agents.model_manager.load_model(model_id, device_map="cuda", quantization_config = nf4_config)

responses_ll = []
for i in range(0,len(images),5):
    image_tags = "".join([f"<|image_{j+1}|>" for j in range(len(images[i:i+5]))])

    promt_a = """
    Analytics Report Request for Drone ULG Data
    I have a set of plots derived from my drone's ULG data. As an expert in drone analytics, I would like you to analyze these plots and provide a detailed report.
    The report should include the following:
    1)Understanding the Plots 2)Impact on Drone Behavior 3)Key Observations 4)Correlations Between Plots
    Rules:
    1)Understand the plots and provide your expert opinion.
    2)Keep the response crisp and summarized, with key observations highlighted.
    3)Highlight any correlations between plots.
    Please ensure the report is comprehensive yet concise, offering actionable insights and a clear understanding of the drone's performance based on the ULG data.
    """

    messages = [ 
        {"role": "user", "content":f"{image_tags}i would like a detailed analysis of the images I have provided, focusing on the metrics displayed and their impact on drone behavior"}, 
        {"role": "assistant", "content": "understand what is present in images and give very insightfull responses as per user questions"}, 
        {"role": "user", "content": f"{promt_a}"} 
    ] 

    # image = Image.open(requests.get(url, stream=True).raw) 

    batch_images = images[i:i+5]
    prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    for j, image in enumerate(batch_images):
        image.id = j + 1
    inputs = processor(prompt,batch_images, return_tensors="pt").to("cuda:0") 

    generation_args = { 
        "max_new_tokens": 1000, 
        "temperature": 0.0, 
        "do_sample": False, 
    } 

    generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args) 

    # remove input tokens 
    generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
    response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0] 
    responses_ll.append(response)
Analysis = ""
for res in responses_ll:
    Analysis += res

In [ ]:
Analysis.split("\n")

In [ ]:
while True:
    user_query = input("Enter your question or type 'exit' to finish: ")
    if user_query.lower() == 'exit':
        break
    prompt_Ab = f"""
    understand the Analytics report completely like An Domain Expert and answer the user Question
    ==========================
    Analysis data:-{Analysis}
    ==========================
    instructions:-
    user questions :-{user_query}"""
    messages = [
        {"role": "system", "content": "intelligent AI system cabale of Answering any question you have"},  # Passing previous responses as memory
        {"role": "user", "content": f"{prompt_Ab}"}
    ]

    prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(prompt, return_tensors="pt").to("cuda:0")

    generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args)
    generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]  # Remove input tokens
    response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    print("AI Response:", response)

In [ ]:
from agent2 import search_a

In [ ]:
while True:
    user_query = input("Enter your question or type 'exit' to finish: ")
    if user_query.lower() == 'exit':
        break
    cosine_similarity, indices = search_a(user_query, 10)
    contextlist = Analytics_data.iloc[indices[0]]["parameter"]
    # context = combine_context(contextlist)
    Agents.create_and_save_plots(df,contextlist.to_list(),user_query)
    
    image_tags = "".join([f"<|image_{j+1}|>" for j in range(len(contextlist.to_list()))])
    path_to_load = input("give the path to directory")
    images_n = Agents.load_images_from_folder(path_to_load)
    promt_a = """
    Analytics Report Request for Drone ULG Data
    I have a set of plots derived from my drone's ULG data. As an expert in drone analytics, I would like you to analyze these plots and provide a detailed report.
    The report should include the following:
    1)Understanding the Plots 2)Impact on Drone Behavior 3)Key Observations 4)Correlations Between Plots
    Rules:
    1)Understand the plots and provide your expert opinion.
    2)Keep the response crisp and summarized, with key observations highlighted.
    3)Highlight any correlations between plots.
    Please ensure the report is comprehensive yet concise, offering actionable insights and a clear understanding of the drone's performance based on the ULG data.
    """

    messages = [ 
        {"role": "user", "content":f"{image_tags}\ni would like a detailed analysis of the images I have provided, focusing on the metrics displayed and their impact on drone behavior"}, 
        {"role": "assistant", "content": "understand what is present in images and give very insightfull responses as per user questions"}, 
        {"role": "user", "content": f"{promt_a}"} 
    ] 

    # messages = [
    #     {"role": "system", "content": f"{Analysis}"},  # Passing previous responses as memory
    #     {"role": "user", "content": f"{user_query}"}
    # ]

    prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = processor(prompt,images_n, return_tensors="pt").to("cuda:0")

    generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args)
    generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]  # Remove input tokens
    response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    print("AI Response:", response)

In [ ]:
df = pd.read_csv("/home/uav/Documents/AI_Hunter/LLMS/total_data.csv")

In [ ]:
df

In [ ]:
df[df["filename"]=="wikipedia"]["synopsis"].to_list()

In [ ]:
def get_embeddings(query):
    embed_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    embeddings = embed_model.encode(query)
    torch_embeddings = torch.tensor(embeddings, dtype=torch.float32)
    norm_embeddings  = F.normalize(torch_embeddings, p=2.0, dim=-1)
    return norm_embeddings

In [ ]:
df["content"].to_list()

In [ ]:
e = get_embeddings(df["content"].to_list())

In [ ]:
d = pd.DataFrame(e)

In [ ]:
df1 = pd.concat([d,df],axis=1)

In [ ]:
df1.to_csv("/home/uav/Documents/AI_Hunter/LLMS/ulg_data/wind1_context.csv",index=False)

In [ ]:
df1

In [ ]:
from agent2 import Agents

In [ ]:
text = Agents.Analytics_one("i want scenarios that involve very high wind conditions. i wan to test my system more in the context of wind so make sure i cover all edge cases in that conext")

In [ ]:
non_empty_columns

In [ ]:
df = pd.read_csv("/home/uav/Documents/AI_Hunter/LLMS/ulg_data/wind1_filtered.csv")

In [ ]:
df.columns.to_list()

In [ ]:
non_empty_columns = [col for col in df.columns if df[col].notnull().any()]
responses = []
columns_l = []
for i in range(0,len(non_empty_columns),250):
    #{Analytics_list[0][1]}
    prompt_t= f"""
    here are analytics i need for my root cause Analytics and
    "Analytics Topics"  = {text}
    ===============================
    "ulg_data columns":-{non_empty_columns[i:i+250]}
    and here are ulg data columns of my drone now tell me all what are columns required for above metrics
    
    rules:
    1)I have Given you "Analytics Topics" i need you to look contextually relevent "ulg_data columns" so that i can make better analytics of this data
    2)Select only the columns that will enhance the quality and accuracy of the analytics.
    3)Ensure the selection is comprehensive but not redundant.
    4) just give column names no need for description and column names as same as given input
    """
    # print(prompt_t)
    messages = [ 
    {"role": "user", "content": "I am an sUAS Software designer and I need your assistance on Automating UAV testing"}, 
    {"role": "assistant", "content": "I am an AI system capable of answering any query you have"}, 
    {"role": "user", "content": prompt_t} ]

    temp = 0.0
    sample = False
    tokens = 1000
    agent_response = Agents.model_x(messages, temp, sample,tokens,"microsoft/Phi-3-medium-128k-instruct")
    responses.append(agent_response[-1]["content"])
    torch.cuda.empty_cache()
    # return agent_response[-1]["content"]


In [ ]:
responses[1].split("\n")

In [ ]:
import re
for i in range(len(responses)):
    pattern = r'-\s*(\S+)'
    extracted_values = re.findall(pattern, responses[i])
    columns_l.append(extracted_values)

In [ ]:
columns_l

In [ ]:
columns_l = [[item.strip("'") for item in sublist] for sublist in columns_l]#cleaning strings
flattened_data = [item for sublist in columns_l for item in sublist]#flatten the list
filtered_data = [item for item in flattened_data if item in non_empty_columns]

In [ ]:
Agents.create_and_save_plots(df,filtered_data,"i want scenarios that involve very high wind conditions. i wan to test my system more in the context of wind so make sure i cover all edge cases in that conext")

In [ ]:
filtered_data_r = [item for item in flattened_data if item+"[0]"  in non_empty_columns]

In [ ]:
for i in flattened_data:
    if i+"[0]" in non_empty_columns:
        print(i)

In [ ]:
len(filtered_data)

In [ ]:
len(filtered_data),len(flattened_data)

In [ ]:
# df1 = pd.read_csv("/home/uav/Documents/AI_Hunter/LLMS/ulg_data/wind1_filtered.csv")
# df2 = pd.read_csv("sub_para.csv")

In [ ]:
df2["content"] = df2["parameter"] +"\n description:- "+df2["description"]

In [ ]:
df2

In [ ]:
filtered_df2 = df2[df2['parameter'].isin(l)]

In [ ]:
normalized_df2 = normalized_df2.reset_index(drop=True)


In [ ]:
for i in normalized_df2["parameter"].to_list():
    if i not in df1.columns.to_list():
        print(i)

In [ ]:
normalized_df2.to_csv("wind1_context.csv",index=False)

In [ ]:
len(normalized_df2["parameter"].to_list())

In [ ]:
counter = 0
l = []
for i in df1.columns.to_list():
    if i not in normalized_df2["parameter"].to_list():
        l.append(i)
        
        

In [ ]:
normalized_df2.dropna("index",axis=1)

In [ ]:
normalized_df2.reset_index()

In [ ]:
for i in range(len(normalized_df2)):
    if normalized_df2["parameter"][i] in ll:
        normalized_df2["parameter"][i] = normalized_df2["parameter"][i]+"[0]"

In [ ]:
for i in normalized_df2

In [ ]:
normalized_df2

In [ ]:
ll = []
for i in normalized_df2["parameter"].to_list():
    if i not in df1.columns.to_list():
        ll.append(i)
        
    

In [ ]:
normalized_df2

In [ ]:
filtered_df2["parameter"].to_list() == df1.columns.to_list()

In [ ]:
df1.head(5)

In [ ]:
df1_t = df1.T

In [ ]:
df1_m = df1_t.loc[l2]

In [ ]:
df_f = df1_m.T

In [ ]:
df1.to_csv("/home/uav/Documents/AI_Hunter/LLMS/ulg_data/wind1_filtered.csv",index=False)

In [ ]:
df2

In [ ]:
csv1_columns = df1.columns
filtered_df2 = df2[df2['parameter'].isin(csv1_columns)]
normalized_df2 = df2[df2['parameter'].isin(normalized_csv1_columns)]

In [ ]:
l = []
for i in normalized_df2["parameter"].to_list():
    if i not in filtered_df2["parameter"].to_list():
        l.append(i)
        
    

In [ ]:
l

In [ ]:
l2 = l + filtered_df2["parameter"].to_list()

In [ ]:
l2

In [ ]:
filtered_df2

In [ ]:
len(normalized_csv1_columns)

In [ ]:
l  = [] 
for i in normalized_csv1_columns:
    if i not in filtered_df2["parameter"].to_list():
        l.append(i+"[0]")
    
    

In [ ]:
l

In [ ]:
len(normalized_csv1_columns)

In [ ]:
len(csv1_columns)

In [ ]:
import re
def normalize_column_name(column_name):
    return re.sub(r'\[.*\]', '', column_name)

normalized_csv1_columns = set(normalize_column_name(col) for col in df1.columns)

In [ ]:
for i in normalized_csv1_columns:
    if i not in filtered_df2["parameter"].to_list():
        print(i)

In [ ]:
df2[df2["parameter"] == "accel_bias"]

In [ ]:
csv1_columns = df1.columns
filtered_df2 = df2[df2['parameter'].isin(csv1_columns)]

In [ ]:
filtered_df2["parameter"]

In [ ]:
df1_t = df1.T

In [ ]:
filtered_df2["parameter"].to_list()[0]

In [ ]:
row_names = df1.index.tolist()

In [ ]:
df1_t.iloc("target_component")

In [ ]:
filtered_rows = df1_t.loc[filtered_df2["parameter"].tolist()]


In [ ]:
filtered_rows

In [ ]:
test1 = df1[csv1_columns]

In [ ]:
for i in normalized_csv1_columns:
    if i not in csv1_columns:
        print(i)

In [ ]:
l = []
for i in df1.columns.to_list():
    if i in df2["parameter"].to_list():
        l.append(i)

In [ ]:
len(l)

In [ ]:
df2

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

torch.random.manual_seed(0)
model_id = "microsoft/Phi-3-medium-128k-instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda", 
    torch_dtype="auto", 
    quantization_config=nf4_config,
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

messages = [
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"},
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])


In [ ]:
user_input = "i want scenarios that involve very high wind conditions. i wan to test my system more in the context of wind so make sure i cover all edge cases in that conext"
mission_data = pd.read_csv(f"user_questions/{user_input}_1.csv")
# df1 = pd.read_csv("/home/uav/Documents/AI_Hunter/LLMS/LLM_simulation/Test the flight stability of my sUAS during river search and rescue operation_2.csv")

In [ ]:

import re
def extract_scenarios(text):
    pattern = r'(Scenario \d+:\n\n[\s\S]*?)(?=Scenario \d+:|$)'
    matches = re.findall(pattern, text)
    scenarios = [match.strip() for match in matches]
    return scenarios

In [ ]:
t = extract_scenarios(mission_data["Senarios"][0])

In [ ]:
t[1].split("\n")

In [ ]:
df1["environment"][0].split("\n")

In [ ]:
data["Senarios"][0].split("\n")

In [ ]:
from agent2 import Agents

In [ ]:
text = Agents.Analytics_one("i want scenarios that involve very high wind conditions. i wan to test my system more in the context of wind so make sure i cover all edge cases in that conext")

In [ ]:
text.split("\n")

In [ ]:
i want to test wind profile

In [ ]:
import pandas as pd
df = pd.read_csv("sub_para.csv")

In [ ]:
s = "".join(data["parameter"].to_list())

In [ ]:
s

In [ ]:
import torch
import re
torch.cuda.empty_cache()
Analytics_list = []
# pattern = 'Scenario 1(\d+):\n([\s\S]*?)\n\n'
pattern = r'(Scenario \d+: [\s\S]*?)(?=Scenario \d+:|$)'

matches = re.findall(pattern, text)

for match in matches:
    Analytics_list.append(match)

In [ ]:
Analytics_list[0].split("\n")

In [ ]:
matches = re.findall(pattern, text)
matches

In [ ]:
matches = re.findall(pattern, text, re.DOTALL)
scenarios = {f'Scenario {match[0]}': match[1].strip() for match in matches}

In [ ]:
scenarios

In [ ]:
import torch
import torch.nn.functional as F
import faiss

from sentence_transformers import SentenceTransformer
import pandas as pd
from transformers import BitsAndBytesConfig
from agent import Agents#,search_a
from transformers import AutoModelForCausalLM ,AutoTokenizer,AutoProcessor,pipeline

In [ ]:
user_input = "i want test wind profile of system so that i cover all edge cases in the context"
scenario_response,cotext = Agents.scenario_generator_Agent(user_input)
torch.cuda.empty_cache()
missions, rest = Agents.helper_for_mission_and_environment(scenario_response)

mission_responses = []
environment_responses = []

# for mission in missions:
    # mission_response = Agents.Mission_Agent(mission,mission_type)
    # mission_responses.append(mission_response)
    # torch.cuda.empty_cache()

# for rest_part in rest:
#     environment_response = Agents.Environment_specification_Agent(rest_part)
#     environment_responses.append(environment_response)
#     torch.cuda.empty_cache()
# environment_json_list, mission_json_list,index = Agents.json_extraction(environment_responses, mission_responses)
# try:
#     df1 = pd.DataFrame([user_input,cotext,scenario_response,str(index)])
#     df1 = df1.T
#     df1.columns = ["question","context","Senarios","index"]
#     df2 = pd.DataFrame([mission_json_list,environment_json_list])
#     df2 = df2.T
#     df2.columns =["misson","environment"]
#     df1.to_csv(f"{user_input}_1.csv",index=False)
#     df2.to_csv(f"{user_input}_2.csv",index=False)
# except Exception as e:
#     print(e)

In [ ]:
missions[0]

In [ ]:
mission_response = Agents.Mission_Agent(missions[0],"px4")
mission_responses.append(mission_response)
torch.cuda.empty_cache()

In [ ]:
rest[0]

In [ ]:
environment_response = Agents.Environment_specification_Agent(rest[0])
environment_responses.append(environment_response)
torch.cuda.empty_cache()

In [ ]:
mission_responses[0].split("\n")

In [ ]:
environment_responses[0]

In [ ]:
environment_json_list, mission_json_list,index = Agents.json_extraction(environment_responses, mission_responses)

In [ ]:
def search_a(query,k):
    query_embeddings = get_embeddings(query)
    print(query_embeddings.shape)
    query_embeddings = query_embeddings.unsqueeze(0)
    cos_sim,cos_indices = index_a.search(query_embeddings.numpy(),k)
    return cos_sim,cos_indices
def get_embeddings(query):
    embed_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    embeddings = embed_model.encode(query)
    torch_embeddings = torch.tensor(embeddings, dtype=torch.float32)
    norm_embeddings  = F.normalize(torch_embeddings, p=2.0, dim=-1)
    
    return norm_embeddings

In [ ]:
Analytics_data = pd.read_csv("Analytics_knowledge.csv")

knowledge_a = Analytics_data.drop(["columns","Unnamed: 0"],axis=1)
knowledge_tensor_a = torch.tensor(knowledge_a.values, dtype=torch.float32)
normalized_embeddings_a  = F.normalize(knowledge_tensor_a, p=2.0, dim=1)
dimension_a = knowledge_tensor_a.shape[1]
print(dimension_a)
res_a = faiss.StandardGpuResources()
index_a = faiss.GpuIndexFlatIP(res_a,dimension_a)
index_a.add(normalized_embeddings_a.numpy())

In [ ]:
knowledge_a

In [ ]:
user_query = input("Enter your question or type 'exit' to finish: ")

cosine_similarity, indices = search_a(user_query, 5)
contextlist = Analytics_data.iloc[indices[0]]["columns"]
context = combine_context(contextlist)

In [ ]:
contextlist,cosine_similarity

In [ ]:
len(contextlist.to_list())

In [ ]:
len(non_empty_columns)

In [ ]:
embed_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = embed_model.encode(non_empty_columns)

In [ ]:
df = pd.DataFrame(embeddings)

In [ ]:


images = Agents.load_images_from_folder("/home/uav/Documents/AI_Hunter/LLMS/LLM_simulation/plots_data")


In [ ]:
batch_images = images[i+1:i+6]


In [ ]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True)#,llm_int4_threshold=6.0,llm_int8_skip_modules=None,trust_remote_code=True)
        
model_id = "microsoft/Phi-3-vision-128k-instruct" 
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda", quantization_config=quantization_config,trust_remote_code=True)

# cls.model =  AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda", trust_remote_code=True, torch_dtype="auto", _attn_implementation='flash_attention_2') # use _attn_implementation='eager' to disable flash attention

processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True) 

images = Agents.load_images_from_folder("/home/uav/Documents/AI_Hunter/LLMS/LLM_simulation/plots_data")

responses_ll = []
for i in range(0,len(images),5):
 
    promt_a = """based on these images give me detailed Analytics Report so that i get better understanding on my drone mission
    and also explain in the context of drone what are these metrics 
    """

    messages = [ 
        {"role": "user", "content": "<|image_1><|image_2|><|image_3|><|image_4|><|image_5|>\nWhat is shown in this image?"}, 
        {"role": "assistant", "content": "The chart displays the percentage of respondents who agree with various statements about their preparedness for meetings. It shows five categories: 'Having clear and pre-defined goals for meetings', 'Knowing where to find the information I need for a meeting', 'Understanding my exact role and responsibilities when I'm invited', 'Having tools to manage admin tasks like note-taking or summarization', and 'Having more focus time to sufficiently prepare for meetings'. Each category has an associated bar indicating the level of agreement, measured on a scale from 0% to 100%."}, 
        {"role": "user", "content": f"{promt_a}"} 
    ] 

    # image = Image.open(requests.get(url, stream=True).raw) 
    
    batch_images = images[i+1:i+6]
    prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    for j, image in enumerate(batch_images):
        image.id = j + 1
    inputs = processor(prompt,batch_images, return_tensors="pt").to("cuda:0") 

    generation_args = { 
        "max_new_tokens": 500, 
        "temperature": 0.0, 
        "do_sample": False, 
    } 

    generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args) 

    # remove input tokens 
    generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
    response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0] 
    responses_ll.append(response)

In [ ]:
batch_images[0]

In [ ]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True) 

model_id = "microsoft/Phi-3-vision-128k-instruct"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda", quantization_config=quantization_config, trust_remote_code=True)
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

images = Agents.load_images_from_folder("/home/uav/Documents/AI_Hunter/LLMS/LLM_simulation/plots_data")
responses_ll = []

for i in range(0, len(images), 5):
    promt_a = """Based on these images, give me a detailed Analytics Report so that I get a better understanding of my drone mission
    and also explain in the context of the drone what these metrics are.
    """
    
    # Create messages with the correct image tags
    image_tags = "".join([f"<|image_{j+1}|>" for j in range(len(images[i:i+5]))])
    messages = [
        {"role": "user", "content": f"{image_tags}\nWhat is shown in these images?"},
        {"role": "assistant", "content": "The chart displays the percentage of respondents who agree with various statements about their preparedness for meetings. It shows five categories: 'Having clear and pre-defined goals for meetings', 'Knowing where to find the information I need for a meeting', 'Understanding my exact role and responsibilities when I'm invited', 'Having tools to manage admin tasks like note-taking or summarization', and 'Having more focus time to sufficiently prepare for meetings'. Each category has an associated bar indicating the level of agreement, measured on a scale from 0% to 100%."},
        {"role": "user", "content": f"{promt_a}"}
    ]

    prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    batch_images = images[i:i+5]
    
    # Process the images and prompt
    inputs = processor(prompt, batch_images, return_tensors="pt").to("cuda:0")

    generation_args = {
        "max_new_tokens": 500,
        "temperature": 0.0,
        "do_sample": False,
    }

    generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args)

    # Remove input tokens
    generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
    response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    responses_ll.append(response)

In [ ]:
len(responses_ll)

In [ ]:
for i in range(0,30,5):
    s = ""
    for j in range(i,i+5):
        s+= f"<|image_{j+1}|>"
    print(s)

In [ ]:
image_tags

In [ ]:
df

In [ ]:
df["columns"] = non_empty_columns

In [ ]:
df.to_csv("Analytics_knowledge.csv")

In [ ]:
data = pd.read_csv("/home/uav/Documents/AI_Hunter/LLMS/ulg_data/test1.csv")

In [ ]:
data.columns

In [ ]:
def find_non_empty_columns(df):
    non_empty_columns = [col for col in df.columns if df[col].notnull().any()]
    return non_empty_columns

# Find non-empty columns in the DataFrame
non_empty_columns = find_non_empty_columns(data)
print("Non-empty columns:", non_empty_columns)

In [ ]:
for i in range(0,len(non_empty_columns),200):
    print("=================================")
    print(i,non_empty_columns[i:i+250])

In [ ]:
from agent import Agents
import re

In [ ]:
i = [1,2,3]

In [ ]:
i[0:10]

In [ ]:
text = Agents.Analytics_one("/home/uav/Documents/AI_Hunter/LLMS/LLM_simulation/i want to test in high temparature and  no wind involvement senarios_1.csv")

        



In [ ]:
import re
Analytics_list = []
pattern = 'Scenario (\d+):\n([\s\S]*?)\n\n'
matches = re.findall(pattern, text)

for match in matches:
    Analytics_list.append(match)

In [ ]:
len(matches)

In [ ]:
responses = []
for i in range(0,len(non_empty_columns),250):
            #{Analytics_list[0][1]}
    prompt_t= f"""
    here are analytics i need for my root cause Analytics and
    "Analytics Topics"  = {Analytics_list[0][1]}
    ===============================
    "ulg_data columns":-{non_empty_columns[i:i+250]}
    and here are ulg data columns of my drone now tell me all what are columns required for above metrics
    rules:
    1)I have Given you "Analytics Topics" i need you to book contextually relevent "ulg_data columns" so that i can make better analytics of this data
  
    """
    print(prompt_t)
    messages = [ 
    {"role": "user", "content": "I am an sUAS Software designer and I need your assistance on Automating UAV testing"}, 
    {"role": "assistant", "content": "I am an AI system capable of answering any query you have"}, 
    {"role": "user", "content": prompt_t} ]

    temp = 0.0
    sample = False
    tokens = 1000
    agent_response = Agents.model_x(messages, temp, sample,tokens)
    responses.append(agent_response[-1]["content"])
    # 2) pick data columns that is 100% relavent to context given above
    #  2) Make sure to use only columns above as if you makeup new names that will fail my pipeline

In [ ]:
responses[2].split("\n")

In [ ]:
columns_l = []
for i in range(len(responses)):
    pattern = r'-\s*(\S+)'
    extracted_values = re.findall(pattern, responses[i])
    columns_l.append(extracted_values)
columns_l = [[item.strip("'") for item in sublist] for sublist in columns_l]
flattened_data = [item for sublist in columns_l for item in sublist]
filtered_data = [item for item in flattened_data if item in non_empty_columns]


In [ ]:
filtered_data

In [ ]:
Analytics_list[0][1]

In [ ]:
len(responses)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
i
def create_and_save_plots(data,filtered_data, output_folder):

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for column in filtered_data:
        plt.figure()
        plt.plot(data.index, data[column], label=column)
        plt.xlabel('Index')
        plt.ylabel(column)
        plt.title(f'Plot of {column}')
        plt.legend()
        plt.grid(True)
        plot_path = os.path.join(output_folder, f'{column}.png')
        plt.savefig(plot_path)
        plt.close()
        print(f'Saved plot for {column} at {plot_path}')

In [ ]:
create_and_save_plots(data,filtered_data,"plots_data")

In [ ]:
t = ["The image contains four separate line plots, each representing a different metric measured by a drone. The metrics are 'temperature_accel', 'signal_quality', 'v_xy_valid', and 'temperature_gyro'. Each plot has an 'Index' on the x-axis and a numerical value on the y-axis, which likely represents the percentage of agreement or a similar metric. The plots show fluctuations in the data points, with some peaks and troughs indicating variations in the measured values over the course of the drone's mission.\n\n\n1. 'temperature_accel' plot: This plot shows a line graph with sharp peaks and valleys, suggesting that the drone's temperature acceleration readings vary significantly throughout its mission. The peaks could indicate moments of rapid acceleration, while the valleys may represent periods of slower movement or deceleration.\n\n2. 'signal_quality' plot: This plot has a relatively flat line close to the 100% mark, indicating that the signal quality remains consistently high throughout the mission. A slight dip to 96% suggests a minor drop in signal quality, which could be due to interference or signal loss.\n\n3. 'v_xy_valid' plot: This plot also shows a flat line, but at a lower percentage value of 100%. This suggests that the drone's positional accuracy in the x and y axes is always valid, with no instances of invalid data.\n\n4. 'temperature_gyro' plot: This plot has a similar pattern to the 'temperature_accel' plot, with sharp peaks and valleys, indicating that the drone's gyroscope readings vary. The peaks could represent moments of rapid change in orientation, while the valleys may indicate stable or unstable conditions.\n\n\nAs a drone expert, I would advise closely monitoring these metrics to ensure the drone's performance and safety. The 'signal_quality' and 'v_xy_valid' plots are particularly important for maintaining communication and navigation accuracy. The 'temperature_accel' and 'temperature_gyro' plots are crucial for understanding the drone's movement dynamics and stability. Any significant deviations in these metrics could indicate potential issues that may affect the drone's operation", "The image contains three separate line plots, each representing a different metric related to drone preparedness for meetings. The metrics are 'power_input_valid', 'z_global', and 'xy_global', as well as 'xy_reset_counter'. Each plot has a horizontal axis labeled 'Index' and a vertical axis with a scale from 0 to 1.04, indicating the level of agreement or performance. The plots show a single line for each metric, suggesting a consistent level of agreement or performance across the data points.\n\n\n1. 'power_input_valid' plot: This plot likely represents the percentage of respondents who agree that they have clear and pre-defined goals for meetings. The line is flat at the 1.00 mark, indicating a 100% agreement level. This suggests that all respondents are in full agreement with having clear and predefined goals for meetings.\n\n2. 'z_global' plot: This plot could be related to the understanding of the drone's exact role and responsibilities when invited to a meeting. The line is flat at the 1.00 mark, indicating a 100% agreement level. This suggests that all respondents are in full agreement with understanding their exact role and responsibilities.\n\n3. 'xy_global' plot: This plot might be about the understanding of the drone's exact role and responsibilities when invited to a meeting. The line is flat at the 1.00 mark, indicating a 100% agreement level. This suggests that all respondents are in full agreement with understanding their exact role and responsibilities.\n\n4. 'xy_reset_counter' plot: This plot could be related to the drone's ability to manage admin tasks like note-taking or summarization. The line is flat at the 1.00 mark, indicating a 100% agreement level. This suggests that all respondents are in full agreement with having more focus time to prepare for meetings.\n\n5. 'xy_valid' plot: This plot likely represents the percentage of respondents who agree that they have tools to manage admin tasks like note-taking or summarization. The line is flat at the 1.00 mark, indicating a 100% agreement level. This suggests", "The image contains three separate line plots, each representing a different metric measured by a drone. The first plot, labeled 'z_reset_counter', shows a horizontal line at the 0.00 mark, indicating no variance or change over the 16,000 index points. The second plot, 'v_z_valid', also displays a horizontal line at the 1.00 mark, suggesting a consistent value across the 16,000 index points. The third plot, 'temperature', has a horizontal line at the 13.48 mark, with minor fluctuations around this value across the 8,000 index points. The fourth plot, 'v_fov', shows a horizontal line at the 0.52 mark, with no variation over the 5,000 index points. The fifth plot, 'visual_odometry_timestamp_rel', has a horizontal line at the 33,500 mark, with slight variations around this value across the 5,000 index points.\n\nAs a drone expert, I would analyze these plots to understand the stability and consistency of the drone's performance metrics. The 'z_reset_counter' and 'v_z_valid' plots suggest that the drone's reset and validation processes are stable and unchanged over time. The 'temperature' plot indicates that the drone maintains a consistent temperature, which is crucial for optimal performance and avoiding overheating. The 'v_fov' plot indicates that the drone's field of view remains constant, ensuring consistent coverage during the mission. The 'visual_odometry_timestamp_rel' plot shows that the drone's visual odometry system maintains a stable relative timestamp, which is important for accurate navigation and tracking.\n\nThese metrics impact the drone's behavior by ensuring consistent performance and reliability. The stable metrics suggest that the drone is operating as expected, with no significant deviations that could affect its mission. However, the slight variations in the 'temperature' and 'visual_odometry_timestamp_rel' plots may require further investigation to determine if they are within acceptable limits or if they indicate potential issues that could impact the drone's performance.\n\nIn terms of drone behavior, the stable metrics"]

In [ ]:
t[2].split("\n")

In [ ]:
user_query = input("Enter your question or type 'exit' to finish: ")
cosine_similarity, indices = search_a(user_query, 5)
contextlist = Analytics_data.iloc[indices[0]]["columns"]
context = combine_context(contextlist)